# Tipi Algebrici (Record e Variant)

In questo capitolo saranno illustrate **XXXXXXXXXXXXXXXXXXXX** del linguaggio OCaml, in particolare relative al paradigma di programmazione funzionale, che quello su cui OCaml si basa principalmente. Il linguaggio prevede comunque aspetti di programmazione imperativa e a oggetti che vedremo successivamente.

### Riferimenti

Questo capitolo si basa su materiale ed esempi di codice tratti da:

- <A href="https://dev.realworldocaml.org/">Real World OCaml</A> (Versione 2), di Yaron Minsky, Anil Madhavapeddy e Jason Hickey

La licenza per l'utilizzo del testo e degli esempi di codice è disponibile nel sito indicato.

Inoltre, alcuni aspetti del linguaggio possono essere approfonditi consultando il manuale ufficiale di OCaml, disponibile a questo indirizzo:

- <A href="https://ocaml.org/manual/index.html">https://ocaml.org/manual/index.html</A>

## Definire nuovi tipi

In OCaml è possibile definire nuovi tipi di dato usando il costrutto `type`

In [1]:
type data = int*int*int ;;

type data = int * int * int


Abbiamo definito un alias per `int*int*int`, che rende più leggibile il codice...

In [2]:
let controlla_data (d:data) =
    match d with
    | gg,mm,aaaa -> gg>0 && gg<=31
                 && mm>0 && mm<=12
                 && aaaa>1900 && aaaa<=2030 ;;

val controlla_data : data -> bool = <fun>


In [3]:
let accoda_data (d:data) lis =
    if controlla_data(d) then d::lis else lis ;;

val accoda_data : data -> data list -> data list = <fun>


## Tipi algebrici

OCaml consente di *comporre* tipi di dato per creare nuovi tipi

Immaginiamo un tipo come un insieme di valori
* `int` come l'insieme dei numeri interi, 
* `bool` come l'insieme {`true`,`false`}, 
* ecc...

Le operazioni di costruzione di nuovi tipi in OCaml sono ispirate alle operazioni insiemistiche: 
* *prodotto cartesiano* ($\times$), a cui corrispondono le *tuple* e i *record*
* *unione* ($\cup$) (che è una *somma* di insiemi), a cui corrispondo i *variant*

## Tipi prodotto: tuple e record

Il prodotto cartesiano di due insiemi (es. $\mathbf{N} \times \mathbf{N}$) è un insieme di coppie.
* $(7,4) \in \mathbf{N}\times \mathbf{N}$

Allo stesso modo il prodotto di due tipi (es. `int * int`) è un tipo di coppie
* `(7,4) : int*int`

Il discorso si estende a tuple di qualunque dimensione, come abbiamo già visto...

## Record

I record sono un modo più avanzato di definire tipi prodotto
* sono simili a tuple, ma possiamo dare un nome agli elementi (detti *campi*)

In [29]:
type punto_2d = { x: float; y: float } ;;

type punto_2d = { x : float; y : float; }


Definito il tipo di un record, possiamo creare valori di quel tipo

In [5]:
let p = { x = 3.; y = -4. } ;;

val p : punto_2d = {x = 3.; y = -4.}


Un valore di tipo record può essere decomposto usando il *pattern matching*

Ad esempio, calcoliamo il quadrante del piano cartesiano in cui ricade un punto

In [26]:
let quadrante {x = x_pos; y = y_pos } =
    match x_pos>=0.,y_pos>=0. with
    | true,true -> 1
    | false,true -> 2
    | false,false -> 3
    | true,false -> 4 ;;

quadrante {x=(-3.); y=2.};;

val quadrante : punto_2d -> int = <fun>


- : int = 2


<center><img src="files/images/quadranti.gif" width="400"></center>

**ATTENZIONE:** Nell'esempio ci sono due pattern matching. Quello che decompone il record è implicito nel `let` e usa il pattern `{x = x_pos; y = y_pos }`

Un po' più semplicemente, possiamo usare direttamente `x` e `y` come nomi di variabili

In [7]:
let quadrante {x; y} =
    match x>=0.,y>=0. with
    | true,true -> 1
    | false,true -> 2
    | false,false -> 3
    | true,false -> 4 ;;

val quadrante : punto_2d -> int = <fun>


Oppure possiamo accedere ai campi usando la *dot notation*: `p.x` e `p.y`

In [8]:
let quadrante p =
    match p.x>=0.,p.y>=0. with
    | true,true -> 1
    | false,true -> 2
    | false,false -> 3
    | true,false -> 4 ;;

val quadrante : punto_2d -> int = <fun>


La dot notation consente di usare più variabili di tipo record in modo semplice

In [32]:
let p = {x=3.; y=3.} ;;
let q = {x=6.; y=7.} ;;
p.x ;;

val p : punto_2d = {x = 3.; y = 3.}


val q : punto_2d = {x = 6.; y = 7.}


- : float = 3.


Esempio: Distanza tra due punti nel piano ($d = \sqrt{(x_2-x_1)^2+(y_2-y_1)^2}$ )

In [30]:
let distanza p1 p2 =
    sqrt ( (p2.x -. p1.x)**2. +. (p2.y -. p1.y)**2. ) ;;

val distanza : punto_2d -> punto_2d -> float = <fun>


In [33]:
distanza p q ;;

- : float = 5.


Quando si usano record con molti campi, le operazioni possono diventare verbose

In [12]:
type persona = {nome: string; cognome: string; eta: int; indirizzo: string; citta: string} ;;

type persona = {
  nome : string;
  cognome : string;
  eta : int;
  indirizzo : string;
  citta : string;
}


In [13]:
let mario = {nome="Mario"; cognome="Rossi"; eta=45; indirizzo="Via Roma 10"; citta="Pisa"} ;;

val mario : persona =
  {nome = "Mario"; cognome = "Rossi"; eta = 45; indirizzo = "Via Roma 10";
   citta = "Pisa"}


Una piccola semplificazione è che possiamo definire una variabile in funzione di un'altra usando il costrutto di *functional update* `with`

In [14]:
let bianca = { mario with nome = "Bianca"; eta=13 } ;;

val bianca : persona =
  {nome = "Bianca"; cognome = "Rossi"; eta = 13; indirizzo = "Via Roma 10";
   citta = "Pisa"}


## TODO: Digressone: record e oggetti
I record assomigliano a dizionari in JavaScript, e potrebbero sembrare oggetti "assegnando" funzioni agli elementi

La differenza con gli oggetti è che sono immutabili (le variabili non rappresentano uno stato interno... che è una caratteristica di base della programmazione a oggetti)

In [15]:
p.x = 10.;;

- : bool = false


In [16]:
type finto_oggetto = { x: int; update: int -> int } ;;

type finto_oggetto = { x : int; update : int -> int; }


In [17]:
let o = { x = 10; update = fun y -> x + y} ;;

error: compile_error

## Tipi unione (o somma): variant

L'unione di due insiemi è... l'unione di due insiemi!
* $C = A \cup B$
* $x \in C$ se e solo se $x \in A$ oppure $x \in B$

L'unione di due tipi `t1` e `t2` è un nuovo tipo `t` che idealmente "include" tutti i valori di tipo `t1` e `t2`

Mentre in alcuni linguaggi (ad es. TypeScript) si possono unire direttamente due tipi, scrivendo ad esempio:

```
[TYPESCRIPT] type t = string | int
```

l'operazione di unione prevista da OCaml prevede di *etichettare* i valori dei tipi da unire

Ad esempio, per unire i tipi `string` e `int`, dobbiamo scegliere due etichette (ad esempio `Txt` e `Num`) e usarle in questo modo:

In [88]:
type numero_testo =
    | Txt of string
    | Num of int ;;

type numero_testo = Txt of string | Num of int


In sostanza, ogni riga della definizione corrisponde a un caso possibile di tipo di valore, ed è caratterizzato da una etichetta distinta (detta *costruttore*)

**ATTENZIONE:** I costruttori devono necessariamente iniziare con la lettera maiuscola

Anche i valori del nuovo tipo dovranno essere etichettati nello stesso modo:

In [19]:
let x = Txt "34" ;;
let y = Num 26 ;;

val x : numero_testo = Txt "34"


val y : numero_testo = Num 26


Dato un valore, è possibile ricostruire a quale dei tipi che sono stati uniti appartenga usando i costruttori nel pattern matching

In [20]:
match x with
| Txt s -> "testo"
| Num n -> "numero" ;;

- : string = "testo"


Qualche esempio di funzione che usa il tipo `numero_testo`:

In [21]:
let int_of_nt x =
    match x with
    | Txt s -> int_of_string s
    | Num n -> n ;;
    
let string_of_nt x =
    match x with
    | Txt s -> s
    | Num n -> string_of_int n ;;
    
int_of_nt (Txt "4") ;;

val int_of_nt : numero_testo -> int = <fun>


val string_of_nt : numero_testo -> string = <fun>


- : int = 4


Nella dichiarazione di un tipo unione, la parte `of <tipo>` è facoltativa
* si possono definire casi che consistono solo del costruttore
* sono come "singoletti" nell'insiemistica

Usando i soli costruttori, si possono definire *tipi enumerazione*
* In molti linguaggi chiamati *enum*

In [107]:
type giorno = Lun | Mar | Mer | Gio | Ven | Sab | Dom ;;

type giorno = Lun | Mar | Mer | Gio | Ven | Sab | Dom


In [108]:
Gio ;;

- : giorno = Gio


In [103]:
let is_weekend g =
    match g with
    | Sab | Dom -> true
    | _ -> false ;;

val is_weekend : giorno -> bool = <fun>


Un esempio più complesso: 
* tipo `int` esteso con `NaN`, $+ \infty$ e $- \infty$

In [22]:
type int_ext =
    | Num of int
    | NaN
    | Plus_inf
    | Minus_inf ;;

type int_ext = Num of int | NaN | Plus_inf | Minus_inf


Definiamo un po' di operazioni artimetiche sul nuovo tipo

In [52]:
let sum x y =
    match x,y with
    | NaN,_ | _,NaN -> NaN
    | Plus_inf,Minus_inf | Minus_inf,Plus_inf -> NaN
    | Plus_inf,_ | _,Plus_inf -> Plus_inf
    | Minus_inf,_ | _,Minus_inf -> Minus_inf
    | Num n1,Num n2 -> Num (n1+n2) ;;

val sum : int_ext -> int_ext -> int_ext = <fun>


**ATTEZIONE:** nessun warning: il pattern matching è esausivo
* non abbiamo scordato casi!

Ancora:

In [59]:
let div x y =
    match x,y with
    | NaN,_ | _,NaN -> NaN
    | Plus_inf,Plus_inf | Minus_inf,Minus_inf -> NaN
    | Plus_inf,Minus_inf | Minus_inf,Plus_inf -> NaN
    | Plus_inf,_ -> Plus_inf
    | Minus_inf,_ -> Minus_inf
    | _,Plus_inf | _,Minus_inf -> Num 0
    | Num 0,Num 0 -> NaN
    | Num n,Num 0 -> if n>0 then Plus_inf else Minus_inf 
    | Num n1,Num n2 -> Num (n1/n2) ;;

val div : int_ext -> int_ext -> int_ext = <fun>


In [65]:
div (Num 3) (Num 0) ;;

- : int_ext = Plus_inf


$3/0$


In [66]:
sum (Num 5) (div (Num 3) (Plus_inf)) ;;

- : int_ext = Num 5


$5 + (3 \,/ \!+\!\infty)$

### Tipi opzione e tipi polimorfi

Un esempio di tipo variant molto utilizzato (e *built-in* in OCaml) è il tipo opzione
* consente di specificare un valore che può essere assente
* usa i costruttori `Some` e `None`

In [67]:
Some 4;;

- : int option = Some 4


In [69]:
None ;;

- : 'a option = None


I tipi opzione consentono di definire funzioni che restituiscono `None` in casi particolari

In [96]:
let rec massimo lis =
    match lis with
    | [] -> None   
    | x::lis' ->  match massimo lis' with
                    | None -> Some x 
                    | Some max -> if x>max then Some x
                                    else Some max ;;           

val massimo : 'a list -> 'a option = <fun>


In [97]:
massimo [3;4;2] ;;
massimo ["albero";"cane";"bambino"] ;;
massimo [] ;;

- : int option = Some 4


- : string option = Some "cane"


- : 'a option = None


I tipi opzione sono (pre-)definiti in questo modo:

In [85]:
type 'a option =
    | Some of 'a
    | None ;;

type 'a option = Some of 'a | None


questo esempio mostra come sia possibile definire *tipi polimorfi*
* usando le variabili di tipo `'a`, `'b`, ...

## TODO: esempio con tipi variant e record insieme

## Tipi ricorsivi
Un'importante generalizzazione dei tipi unione sono i *tipi ricorsivi*
* sono tipi variant definiti in termini di se stessi

In [124]:
type lista_di_int =
    | Nil
    | Elem of int * lista_di_int ;;
    
let lst = Elem (3 ,Elem (4, Elem (6,Nil))) ;;

type lista_di_int = Nil | Elem of int * lista_di_int


val lst : lista_di_int = Elem (3, Elem (4, Elem (6, Nil)))


In [125]:
let cons x lis =
    Elem (x,lis) ;;

cons 5 lst;;

val cons : int -> lista_di_int -> lista_di_int = <fun>


- : lista_di_int = Elem (5, Elem (3, Elem (4, Elem (6, Nil))))


In [127]:
let rec somma_lista lis = 
    match lis with
    | Nil -> 0
    | Elem (x,lis') -> x + somma_lista lis' ;;

val somma_lista : lista_di_int -> int = <fun>


In [129]:
somma_lista ( Elem (3 ,Elem (4, Elem (6,Nil))));;

- : int = 13


Il tipo built-in `'a list` è un caso particolare di variant ricorsivo polimorfo

In [1]:
type 'a my_list =
   | []
   | Cons of 'a * 'a my_list ;; 

type 'a my_list = [] | Cons of 'a * 'a my_list


In [4]:
[] ;;
Cons (3, Cons (4, Cons (1,[]))) ;; (* corrisponde a 3::4::1 *)
Cons ("a", Cons ("b",[])) ;; (* corrisponde a "a"::"b"::[] *)

- : 'a my_list = []


- : int my_list = Cons (3, Cons (4, Cons (1, [])))


- : string my_list = Cons ("a", Cons ("b", []))


Per semplificare la lettura, OCaml usa una sintassi ad-hoc per i valori di tipo `'a list` 
* `[x1;x2;...]`

### Tipi ricorsivi e... Alberi
Grazie ai tipi ricorsivi è semplice definire alberi

In [29]:
type albero_bin = 
  | Nodo of int*albero_bin*albero_bin 
  | Foglia of int ;;

type albero_bin = Nodo of int * albero_bin * albero_bin | Foglia of int


In [35]:
let n1 = Foglia 4 ;;
let n2 = Foglia 6 ;;
let n3 = Nodo (2,n1,n2) ;;
let n4 = Foglia 8 ;;
let n5 = Nodo (5,n3,n4) ;;

val n1 : albero_bin = Foglia 4


val n2 : albero_bin = Foglia 6


val n3 : albero_bin = Nodo (2, Foglia 4, Foglia 6)


val n4 : albero_bin = Foglia 8


val n5 : albero_bin = Nodo (5, Nodo (2, Foglia 4, Foglia 6), Foglia 8)


In [34]:
let visita_ant a =
    match a with
    | Foglia v -> [v]
    | Nodo (v,sx,dx) -> v::((visita_ant sx)@(visita_ant dx)) ;;
    
visita_ant n5 ;;

val visita_ant : albero_bin -> int list = <fun>


- : int list = [5; 2; 4; 6; 8]


In [40]:
let rec somma_albero a =
    match a with
    | Foglia v -> v
    | Nodo (v,sx,dx) -> v + somma_albero sx + somma_albero dx ;;
    
somma_albero n5 ;;

val somma_albero : albero_bin -> int = <fun>


- : int = 25


### Tipi ricorsivi e... Abstract Syntax Tree (AST)

Tipo di albero utile per gli scopi di questo corso: gli alberi di sintassi astratta

## TODO: Discorso tipi algebrici OCaml vs JS/TS
Anche in TS ci sono le tuple (ma sono in realtà array). Anche in JS ci sono i record (ma sono in realtà array). Anche in TS ci sono i tipi unione (ma sono molto ambigui)